In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/data/tim/heronWorkspace/src")
sys.path.append("/data/tim/heronWorkspace/AE")
sys.path.append("/data/tim/heronWorkspace/classifier")

In [3]:

from AEHeronModelV1 import AEHeronModel
from AEHeronModelV2 import CAEHeron
from lightning.pytorch.callbacks import ModelSummary
from torchsummary import summary
import HeronImageLoader
from torch.utils.data import DataLoader, BatchSampler
from matplotlib import pyplot as plt
import lightning.pytorch as pl
from lightning.pytorch.tuner import Tuner
import pandas as pd
from lightning.pytorch.loggers import CSVLogger
from MLPV1 import MLP, MLPMSEHeatMap
from models import MLPBasic, CAEBigBottleneck, CAESmallBottleneckWithLinear, MLPBasicHeatMap, CAEV1
import numpy as np
import torch.nn.functional as F
import torch
from torchvision.transforms import GaussianBlur
from PIL import Image, ImageFilter
import random
from scipy.stats import loguniform
from MLPDatasets import MLPDatasetThreeConsecutive
# from torchmetrics.image import StructuralSimilarityIndexMeasure
from skimage.metrics import structural_similarity as ssim


# Which cameras
If we take the data of tina dubachs classification as true, there are a lot of birds in the training set without motion-sensor taken pictures

SBU4 isn't representative for training

In [19]:
dfUntilSBU2 = pd.read_csv("/data/tim/heronWorkspace/ImageData/imagePropsSBU2.csv") #contains top 10 cameras
unwanted = dfUntilSBU2.columns[dfUntilSBU2.columns.str.startswith('Unnamed')]
dfUntilSBU2.drop(unwanted, axis=1, inplace=True)
dfUntilSBU2 = dfUntilSBU2[dfUntilSBU2["grayscale"] == "False"]
dfUntilSBU2["cam"] = dfUntilSBU2["ImagePath"].str.split("_").str[1]
dfTinaDubach = pd.read_csv("/data/shared/herons/TinaDubach_data/CameraData_2017_july.csv", encoding='unicode_escape', on_bad_lines="warn", sep=";")

dfAll = pd.merge(dfTinaDubach, dfUntilSBU2, left_on="fotocode", how="right", right_on="ImagePath")

#categorical sort
folders = dfTinaDubach.groupby(["camera"]).size().sort_values(ascending=False)
folders = folders.reset_index()
folders = folders["camera"].tolist()[:10]

dfAll['cam'] = pd.Categorical(dfAll['cam'], folders)
dfAll.sort_values(by=['cam', "motion"])

dfAll = dfAll.drop_duplicates(subset = ["ImagePath"], keep="first") # drop duplicates
dfAll = dfAll[["cam", "motion", "ImagePath", "species"]].groupby(["cam", "motion"]).agg(nrFrames = ("ImagePath", "count"), nrAnomalies = ("species", lambda x: x.notna().sum()))
print("top 10 cameras sorted by nr of frames\nonly rgb frames")
dfAll


top 10 cameras sorted by nr of frames
only rgb frames


nrFrames  nrAnomalies
cam  motion                       
SBU4 False       9095          711
     True        3804          836
SBU1 False       8764           40
     True        1354            6
NEN1 False       9125          206
     True         899           47
SGN1 False       8528          170
     True        2797           51
KBU1 False       9184           53
     True        3290          177
SGN3 False       9387           86
     True        1947          137
SBU3 False       9044          113
     True        1908           71
SGN4 False       8655           66
     True        3675          121
NEN3 False       9270          136
     True        2966           27
SBU2 False       8999           37
     True        2605           26

## pred last to current and next to current

In [ ]:
def applyFiltersAndPlot(imgs, preds, imds):
    f, a = plt.subplots(1,11, figsize=(110,10))


    preDiff = imds[1] - imds[0]
    nextDiff = imds[1] - imds[2]
   
    a[0].imshow(imgs[1])
    a[0].text(-0.5,0.5, f'True: {lbl}', size=100, ha="center", color = "red", transform=a[0].transAxes)
    a[0].text(0.5,-0.2, f'current Image', size=30, ha="center", transform=a[0].transAxes)

    a[1].imshow(preds[1])
    a[1].text(0.5,-0.4, f'prediction\nssim img to pred: {ssim(imgs[0], preds[0], channel_axis=2, data_range=1)}', size=30, ha="center", transform=a[1].transAxes)

    a[2].imshow(np.abs(imds[1]), cmap="hot", interpolation='none')
    a[2].text(0.5,-0.5, f'difference\nimd =|currImg - pred|\nssim prev imd to curr imd:\n{ssim(imds[0], imds[1], data_range=1)}', size=30, ha="center", transform=a[2].transAxes)

    a[3].imshow(np.abs(preDiff), cmap="hot", interpolation='none')
    a[3].text(0.5,-0.2, f'|Imd - ImdPrev|', size=30, ha="center", transform=a[3].transAxes)

    onlyPosDiffPred = np.where(preDiff < 0, 0, preDiff)

    a[4].imshow(onlyPosDiffPred, cmap="hot", interpolation='none')
    a[4].text(0.5,-0.4, f'(Imd - ImdPrev)\nonly Positives', size=30, ha="center", transform=a[4].transAxes)


    onlyPosDiffNext = np.where(nextDiff < 0, 0, nextDiff)

    posDiffPredAndNext = np.divide(np.add(onlyPosDiffPred, onlyPosDiffNext), 2.0)
    snr = np.mean(posDiffPredAndNext)/np.std(posDiffPredAndNext)
    a[5].imshow(posDiffPredAndNext, cmap = "hot", interpolation='none')
    a[5].text(0.5,-0.5, f'only Positives:\n((Imd - ImdPrev) + (Imd - ImdNext)\nsum: {np.sum(posDiffPredAndNext)}\nsnr: {snr}', size=30, ha="center", transform=a[5].transAxes)

    posDiffPredAndNextPil = Image.fromarray(np.uint8(posDiffPredAndNext * 255))
    
    filtered = posDiffPredAndNextPil.filter(ImageFilter.GaussianBlur(0.01))
    a[6].imshow(filtered, cmap = "hot", interpolation='none')
    a[6].text(0.5,-0.4, f'blurPil\nsum: {np.sum(filtered)/255}\nsnr: {np.mean(filtered)/np.std(filtered)}', size=30, ha="center", transform=a[6].transAxes)

    filtered = blur.forward(posDiffPredAndNextPil)
    a[7].imshow(filtered, cmap = "hot", interpolation='none')
    a[7].text(0.5,-0.4, f'blurTorch\nsum: {np.sum(filtered)/255}\nsnr: {np.mean(filtered)/np.std(filtered)}', size=30, ha="center", transform=a[7].transAxes)

    filtered = np.where(posDiffPredAndNext < np.max(posDiffPredAndNext) * 0.3, 0, posDiffPredAndNext)
    a[8].imshow(filtered, cmap = "hot", interpolation='none')
    a[8].text(0.5,-0.4, f'cutoff 0.3 * max\nsum: {np.sum(filtered)/255}\nsnr: {np.mean(filtered)/np.std(filtered)}', size=30, ha="center", transform=a[8].transAxes)

    filtered = posDiffPredAndNextPil.filter(ImageFilter.SMOOTH)
    a[9].imshow(filtered, cmap = "hot", interpolation='none')
    a[9].text(0.5,-0.4, f'smooth\nsum: {np.sum(filtered)/255}\nsnr: {np.mean(filtered)/np.std(filtered)}', size=30, ha="center", transform=a[9].transAxes)

    filtered = posDiffPredAndNextPil.filter(ImageFilter.MedianFilter)
    a[10].imshow(filtered, cmap = "hot", interpolation='none')
    a[10].text(0.5,-0.4, f'median\nsum: {np.sum(filtered)/255}\nsnr: {np.mean(filtered)/np.std(filtered)}', size=30, ha="center", transform=a[10].transAxes)

    plt.show()


def heatMap(before: torch.Tensor, after: torch.Tensor, stepY, stepX, loss_fn = F.mse_loss):
    heatMap = []
    for i in range(0, before.shape[-2]-stepY+1, stepY):
        row = []
        for j in range(0, before.shape[-1]-stepX+1, stepX):
            row.append(loss_fn(before[:, i:i+stepY, j:j+stepX], after[:, i:i+stepY, j:j+stepX]).item())
        heatMap.append(row)
    return torch.tensor(heatMap).type_as(before)


checkPoint = '/data/tim/heronWorkspace/logs/CAEV1/version_3/checkpoints/epoch=49-step=19350.ckpt'
caeLoaded = CAEHeron.load_from_checkpoint(checkPoint, model = CAEV1)
caeLoaded.freeze()
print(caeLoaded.hparams)

dataLoader = DataLoader(MLPDatasetThreeConsecutive(set="test", resize_to=CAEV1.imsize, cameras=caeLoaded.hparams.cameras), batch_size=1, shuffle=True, num_workers=4)
print(f'Length Dataset: {len(dataLoader.dataset.imagePaths)}')
unnorm = HeronImageLoader.UnNormalize()


stepY = 1
stepX = 1

blur = GaussianBlur(kernel_size=5, sigma=1.0)
# ssim = StructuralSimilarityIndexMeasure(reduction="none")

print("Start Loop")

for i, (imArr, lblArr, _) in enumerate(dataLoader):
    lbl = lblArr[0].item()
    imgs = [x.to(caeLoaded.device) for x in imArr] #three consecutive images

    preds = [caeLoaded(x) for x in imgs] #imgs are already of shape (1, 3, 215, 323)
    
    # print(preds)

    imgs = [unnorm(x.squeeze(0)) for x in imgs]
    preds = [unnorm(x.squeeze(0)) for x in preds]

    imdsL1 = [heatMap(img, pred, stepY, stepX, loss_fn = F.l1_loss) for img, pred in zip(imgs, preds)]
    imdsMSE = [heatMap(img, pred, stepY, stepX, loss_fn = F.mse_loss) for img, pred in zip(imgs, preds)]

    imgs = [x.permute(1, 2, 0).cpu().numpy() for x in imgs]
    preds = [x.permute(1, 2, 0).cpu().numpy() for x in preds]
    imdsL1 = [x.cpu().numpy() for x in imdsL1]
    imdsMSE = [x.cpu().numpy() for x in imdsMSE]

    print("L1:")
    applyFiltersAndPlot(imgs, preds, imdsL1)
    print("MSE:")
    applyFiltersAndPlot(imgs, preds, imdsMSE)
    
    if (i > 10):
        break
